In [25]:
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import csv
import shapely
from shapely.geometry import Point
import os 
import time
import json
from dotenv import load_dotenv
import openrouteservice as ors

In [26]:
directory = os.getcwd()

os.chdir(directory)

In [27]:
hdb_centroid_pair_df = pd.read_csv(r"..\data\HDB_Centroid_MRT pairing data.csv")
private_centroid_pair_df = pd.read_csv(r"..\data\Private_Centroid_MRT pairing data.csv")


with open(r'..\data\HDB_MRT_routes.json', 'r') as f:
    hdb_centroid_pair_route = json.load(f)

hdb_centroid_pair_df['route'] = hdb_centroid_pair_route

with open(r'..\data\Private_MRT_routes.json', 'r') as f:
    private_centroid_pair_route = json.load(f)

private_centroid_pair_df['route'] = private_centroid_pair_route

In [28]:
hdb_centroid_pair_df.shape

(275, 16)

In [29]:
private_centroid_pair_df.shape

(275, 16)

In [30]:
route_df = pd.concat([hdb_centroid_pair_df,private_centroid_pair_df],axis = 0).reset_index(drop = True)

In [31]:
route_df.shape

(550, 16)

In [32]:
route_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          550 non-null    int64  
 1   index               550 non-null    int64  
 2   Latitude_x          550 non-null    float64
 3   Longitude_x         550 non-null    float64
 4   index_x             550 non-null    int64  
 5   join_key            550 non-null    object 
 6   MRT Name            550 non-null    object 
 7   Latitude_y          550 non-null    float64
 8   Longitude_y         550 non-null    float64
 9   index_y             550 non-null    int64  
 10  euclidean_distance  550 non-null    float64
 11  coordinate_pair     550 non-null    object 
 12  distance            547 non-null    float64
 13  duration            547 non-null    float64
 14  centroid_name       550 non-null    object 
 15  route               547 non-null    object 
dtypes: float

In [33]:
fig = go.Figure()

In [34]:
for i in range(len(route_df)):
    try:
        print(i)
        route_data = route_df.loc[i, 'route']
        route_coordinates = route_data['features'][0]['geometry']['coordinates']
        lons, lats = zip(*route_coordinates)
        fig.add_trace(go.Scattermapbox(
            mode="lines",
            lon=lons,
            lat=lats,
            marker={'size': 10},
            hoverinfo = 'text',
            text = ("Time taken:" + str(round(route_df['duration'][i]/60,2)) + " min" + '<br>' +
                    "Distance:" + str(round(route_df['distance'][i]/1000,3)) + " km"),
            showlegend= False,
        ))
    except Exception as e:
        print(f"Error processing index {i}: {e}")
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
Error processing index 65: 'NoneType' object is not subscriptable
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260


In [35]:
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'style': "open-street-map",
        'center': {'lat': 1.36, 'lon': 103.85},
        'zoom': 10.55})

fig.show()

In [38]:
mrt_stations_df = pd.read_csv(r"..\data\mrt_stations.csv",usecols = [1,2,3])
mrt_stations_df = mrt_stations_df[~mrt_stations_df['MRT Name'].str.contains('SUB')]
mrt_stations_df.sort_values(by='MRT Name', inplace=True)
mrt_stations_df


,MRT Name,Latitude,Longitude
12,ADMIRALTY MRT STATION,1.440589,103.800991
9,ALJUNIED MRT STATION,1.316433,103.882906
54,ANG MO KIO MRT STATION,1.369429,103.849455
132,BAKAU LRT STATION,1.387994,103.905415
57,BANGKIT LRT STATION,1.380022,103.772647
...,...,...,...
186,WOODLANDS SOUTH MRT STATION,1.427488,103.792730
85,WOODLEIGH MRT STATION,1.339190,103.870818
82,YEW TEE MRT STATION,1.397298,103.747358
84,YIO CHU KANG MRT STATION,1.381499,103.845171


In [39]:
#Group by MRT, calculate the average distance from MRT to each centroid into another dataframe and reset its index
df = route_df.groupby('MRT Name')['distance'].mean()
df = round(df/1000,3)
df = df.reset_index()

df

mrt_stations_df = pd.merge(mrt_stations_df, df, left_on='MRT Name',right_on = "MRT Name",how = "left")

mrt_stations_df

,MRT Name,Latitude,Longitude,distance
0,ADMIRALTY MRT STATION,1.440589,103.800991,0.985
1,ALJUNIED MRT STATION,1.316433,103.882906,0.799
2,ANG MO KIO MRT STATION,1.369429,103.849455,1.212
3,BAKAU LRT STATION,1.387994,103.905415,0.839
4,BANGKIT LRT STATION,1.380022,103.772647,0.583
...,...,...,...,...
209,WOODLANDS SOUTH MRT STATION,1.427488,103.792730,1.135
210,WOODLEIGH MRT STATION,1.339190,103.870818,0.907
211,YEW TEE MRT STATION,1.397298,103.747358,0.875
212,YIO CHU KANG MRT STATION,1.381499,103.845171,2.602


In [40]:
mrt_station_trace = go.Scattermapbox(lat = mrt_stations_df['Latitude'],
                                     lon =mrt_stations_df['Longitude'],
                                     mode = 'markers',
                                     hoverinfo = 'text',
                                     text = (mrt_stations_df['MRT Name'] + '<br>' +
                                             'Latitude: ' + mrt_stations_df['Latitude'].apply(lambda x: f"{x:.1f}").astype(str) + '<br>' +
                                             'Longitude: ' + mrt_stations_df['Longitude'].apply(lambda x: f"{x:.1f}").astype(str) + '<br>'+
                                             'average distance to centroids: ' + mrt_stations_df['distance'].astype(str) + ' km'),
                                     name = 'MRT Stations',
                                     marker_size = 10
                                     )


fig.add_trace(mrt_station_trace)

In [41]:
residential_centroid_trace = go.Scattermapbox(lat = route_df['Latitude_x'],
                                              lon =route_df['Longitude_x'],
                                              mode = 'markers',
                                              name = 'Residential Centroids',
                                              hoverinfo = 'text',
                                              text = (route_df['centroid_name'] + '<br>' +
                                                      'Latitude: ' +route_df['Latitude_x'].apply(lambda x: f"{x:.1f}").astype(str) + '<br>' +
                                                      'Longitude: ' + route_df['Longitude_x'].apply(lambda x: f"{x:.1f}").astype(str) + '<br>')
                                                      )

fig.add_trace(residential_centroid_trace)
